# Compute read co-occurrence data for phasing

In [1]:
%run "Header.ipynb"

In [2]:
import time
import pickle
import pysam
import skbio
from collections import defaultdict
from itertools import combinations
from linked_mutations_utils import find_mutated_positions, gen_ddi

In [3]:
# This probably won't save a noticeable amount of memory, but humor me
i2n = "ACGT"
n2i = {"A": 0, "C": 1, "G": 2, "T": 3}

## 1. For each read, identify all nucleotides aligned to mutated positions spanned by this read

This takes about 1.8 hours for the three selected genomes. (That said, these genomes have super high coverage, so for less-well-covered genomes this will probably go faster.)

In [3]:
bf = pysam.AlignmentFile("../main-workflow/output/fully-filtered-and-sorted-aln.bam", "rb")

t1 = time.time()
for seq in SEQS:
    fasta = skbio.DNA.read("../seqs/{}.fasta".format(seq))
    
    # Identify all mutated positions in this genome up front to save time.
    print(f"Identifying mutated positions in genome {seq2name[seq]}...")
    mutated_positions = find_mutated_positions(seq)
    print(f"Found {len(mutated_positions):,} mutated positions in {seq2name[seq]}.")
    print("Going through these positions...")
    
    # This should already be implicitly sorted, I think, but the code below relies on mutated_positions being
    # in the exact same order as expected. So we may as well be paranoid.
    mutated_positions = sorted(mutated_positions)
    
    # Maps read name to another dict of mutated position -> aligned nucleotide (in A, C, G, T).
    # We build this up all at once so that we can take supplementary alignments of the same read into account.
    readname2mutpos2nt = defaultdict(dict)
    
    # Go through all linear alignments of each read to this genome...
    ts1 = time.time()
    for ai, aln in enumerate(bf.fetch(seq), 1):
        if ai % 1000 == 0:
            print(
                f"\tOn aln {ai:,} in seq {seq2name[seq]}. "
                f"Time spent on {seq2name[seq]} so far: {time.time() - ts1:,.2f} sec."
            )
        ap = aln.get_aligned_pairs(matches_only=True)
        
        # Iterating through the aligned pairs is expensive. Since read lengths are generally in the thousands
        # to tens of thousands of bp (which is much less than the > 1 million bp length of any bacterial genome),
        # we set things up so that we only iterate through the aligned pairs once. We maintain an integer, mpi,
        # that is a poor man's "pointer" to an index in mutated_positions.
        
        mpi = 0
        
        # Go through this aln's aligned pairs. As we see each pair, compare the pair's reference position
        # (refpos) to the mpi-th mutated position (herein referred to as "mutpos").
        #
        # If refpos >  mutpos, increment mpi until refpos <= mutpos (stopping as early as possible).
        # If refpos == mutpos, we have a match! Update readname2mutpos2ismutated[mutpos] based on
        #                      comparing the read to the reference at the aligned positions.
        # If refpos <  mutpos, continue to the next pair.
        
        readname = aln.query_name
        for pair in ap:
            
            refpos = pair[1]
            mutpos = mutated_positions[mpi]
            
            no_mutations_to_right_of_here = False
            
            # Increment mpi until we get to the next mutated position at or after the reference pos for this
            # aligned pair (or until we run out of mutated positions).
            while refpos > mutpos:
                mpi += 1
                if mpi < len(mutated_positions):
                    mutpos = mutated_positions[mpi]
                else:
                    no_mutations_to_right_of_here = True
                    break
            
            # I expect this should happen only for reads aligned near the right end of the genome.
            if no_mutations_to_right_of_here:
                break
            
            # If the next mutation occurs after this aligned pair, continue on to a later pair.
            if refpos < mutpos:
                continue
                
            # If we've made it here, refpos == mutpos!
            # (...unless I messed something up in how I designed this code.)
            if refpos != mutpos:
                raise ValueError("This should never happen!")
                
            # Finally, get the nucleotide aligned to this mutated position from this read.
            readpos = pair[0]
            # (Convert the nucleotide to an integer in the range [0, 3] using n2i)
            readval = n2i[aln.query_sequence[readpos]]
            
            # Record this specific "allele" for this read. We can use this to link alleles that co-occur
            # on the same read.
            readname2mutpos2nt[readname][mutpos] = readval
            
    with open(f"phasing-data/{seq}_readname2mutpos2nt.pickle", "wb") as dumpster:
        dumpster.write(pickle.dumps(readname2mutpos2nt))
        
print(f"Time taken: {time.time() - t1} sec.")

Identifying mutated positions in genome CAMP...
Found 283 mutated positions in CAMP.
Going through these positions...
	On aln 1,000 in seq CAMP. Time spent on CAMP so far: 1.92 sec.
	On aln 2,000 in seq CAMP. Time spent on CAMP so far: 3.68 sec.
	On aln 3,000 in seq CAMP. Time spent on CAMP so far: 5.67 sec.
	On aln 4,000 in seq CAMP. Time spent on CAMP so far: 7.42 sec.
	On aln 5,000 in seq CAMP. Time spent on CAMP so far: 9.63 sec.
	On aln 6,000 in seq CAMP. Time spent on CAMP so far: 12.93 sec.
	On aln 7,000 in seq CAMP. Time spent on CAMP so far: 16.15 sec.
	On aln 8,000 in seq CAMP. Time spent on CAMP so far: 19.39 sec.
	On aln 9,000 in seq CAMP. Time spent on CAMP so far: 22.66 sec.
	On aln 10,000 in seq CAMP. Time spent on CAMP so far: 26.01 sec.
	On aln 11,000 in seq CAMP. Time spent on CAMP so far: 29.29 sec.
	On aln 12,000 in seq CAMP. Time spent on CAMP so far: 32.59 sec.
	On aln 13,000 in seq CAMP. Time spent on CAMP so far: 35.84 sec.
	On aln 14,000 in seq CAMP. Time spent

## 2. Compute the numbers of reads supporting each observed combination of nucleotides at each pair of mutated positions spanned by at least _minSpan_ reads

We could use Hansel to store this data, but I opted to use a custom solution (for now, at least) for a few reasons:

1. Don't need anything fancy -- just need to store this, not use the probabilistic weighting stuff
2. I don't have time right now to learn Hansel's API (I've read through the docs and am still a bit confused)
3. I think we could probably use less storage (e.g. we only need to store one "triangle" of the matrix; as far as I can tell, Hansel treats H\[a, b, i, j\] as independent of H\[b, a, j, i\], which isn't necessary for haplotyping IMO

In [ ]:
t1 = time.time()
for seq in SEQS:
    with open(f"phasing-data/{seq}_readname2mutpos2nt.pickle", "rb") as loadster:
        # NOTE: this won't necessarily include ALL reads aligned to a sequence -- for example, if a read
        # doesn't cover any mutated positions, it will be omitted from the top level of this dict. (This is
        # because these reads won't be useful for linking the mutated positions.)
        readname2mutpos2nt = pickle.load(loadster)
        print(f"{len(readname2mutpos2nt):,} unique reads described in the data for seq {seq2name[seq]}.")

    ts1 = time.time()
    
    # Now we've seen all alignments of each read, we can go through readname2mutpos2nt and compute
    # co-occurrence information (and create a graph, plot stuff, etc.)
    
    # This defaultdict has two levels:
    # OUTER: Keys are sorted (in ascending order) 0-indexed pairs (tuples) of mutated positions. The
    #        inclusion ofa pair of mutated positions in this defaultdict implies that these two mutated
    #        positions were spanned by at least one read. The value of each pair is another defaultdict:
    #
    # INNER: The keys of this inner defaultdict are pairs of integers, each in the range [0, 3].
    #        These represent the 4 nucleotides (0 -> A, 1 -> C, 2 -> G, 3 -> T): the first entry represents
    #        the nucleotide seen at the first position in the pair (aka the position "earlier" in the genome),
    #        and the second entry represents the nucleotide seen at the second position in the pair (aka
    #        the position "later" in the genome). Of course, many bacterial genomes are circular, so "earlier"
    #        and "later" are kinda arbitrary. Anyway, there are 16 possible pairs in one of these defaultdicts,
    #        since there are 4^2 = 16 different possible combinations of two nucleotides (ignoring deletions,
    #        degenerate nucleotides, etc.) That said, I expect in practice only a handful of nucleotide pairs
    #        will be present for a given position pair. The value of each pair in this defaultdict is
    #        an integer representing the frequency with which this pair of nucleotides was observed on a
    #        spanning read at this pair of positions.
    #
    # So, as an example, if we only have two mutated positions in a genome (at 0-indexed positions 100 and 500),
    # and we saw:
    #
    # - 30    reads with an A at both positions
    # - 1,000 reads with an A at position 100 and a T at position 500
    # - 5     reads with a T at position 100 and an A at position 500
    # - 100   reads with a T at both positions
    # - 3     reads with a C at position 100 and a T at position 500
    # - 1     read  with a G at position 100 and a T at position 500
    #
    # ... then pospair2ntpair2freq would look like
    # {
    #     (100, 500): {
    #         {
    #             (0, 0): 30,
    #             (0, 3): 1000,
    #             (3, 0): 5,
    #             (3, 3): 100,
    #             (1, 3): 3,
    #             (2, 3): 1
    #         }
    #     }
    # }
    pospair2ntpair2freq = defaultdict(gen_ddi)
    for ri, readname in enumerate(readname2mutpos2nt, 1):
        if ri % 100000 == 0:
            print(
                f"\tOn read {ri:,} in seq {seq2name[seq]}. "
                f"Time spent on {seq2name[seq]} so far: {time.time() - ts1:,.2f} sec."
            )
        # TODO: see if we can avoid sorting here -- inefficient when done once for every read, maybe?
        mutated_positions_covered_in_read = sorted(readname2mutpos2nt[readname].keys())
            
        for (i, j) in combinations(mutated_positions_covered_in_read, 2):
            
            # We can assume that i and j are sorted because mutated_positions_covered_in_read is sorted:
            # see https://docs.python.org/3.10/library/itertools.html#itertools.combinations. This is
            # guaranteed, but let's be paranoid just in case:
            if j <= i:
                raise ValueError("Something went horribly wrong with combinations()")
                
            # these are integers in the range [0, 3]
            i_nt = readname2mutpos2nt[readname][i]
            j_nt = readname2mutpos2nt[readname][j]
            
            # We know these mutated positions were observed on the same read, and we know the exact nucleotides
            # this read had at both positions -- update this in pospair2ntpair2freq
            pospair2ntpair2freq[(i, j)][(i_nt, j_nt)] += 1
            
            # print(f"Read {readname} has {i2n[i_nt]} at pos {i} and {i2n[j_nt]} at pos {j}.")

    print(f"Finished going through reads in {seq2name[seq]}.")
    
    # We use the file suffix ".pickle" and "wb" based on the conventions described in
    # https://stackoverflow.com/a/40433504 (...which in turn just reference the python docs).
    with open(f"phasing-data/{seq}_pospair2ntpair2freq.pickle", "wb") as dumpster:
        dumpster.write(pickle.dumps(pospair2ntpair2freq))
        
print(f"Time taken: {time.time() - t1} sec.")

348,612 unique reads described in the data for seq CAMP.
	On read 100,000 in seq CAMP. Time spent on CAMP so far: 0.24 sec.
	On read 200,000 in seq CAMP. Time spent on CAMP so far: 0.50 sec.
	On read 300,000 in seq CAMP. Time spent on CAMP so far: 2.26 sec.
Finished going through reads in CAMP.
257,428 unique reads described in the data for seq BACT1.
